In [168]:
from tkinter import *
from tkinter import scrolledtext

In [169]:
from os import listdir, path, getcwd
from collections import defaultdict

In [170]:
from datetime import datetime
import time

In [171]:
from tqdm.notebook import tqdm

In [258]:
# запуск

MyApp().window.mainloop()

In [242]:
class MyApp:
    
    def __init__(self):
        
        self.book_content, self.book_content_lower = Reader().book_content
                
        self.window = Tk()
        self.window.title("Поиск книг")
        
        self.label = Label(self.window, 
            text="Введите ключевые слова (можно без окончаний) или словосочетания через запятую:", 
                           width = 100, height = 2, font = 15)
        self.label_request_title = Label(self.window, text="", fg="blue", 
                           width = 80, height = 1, font = 15)
        self.label_request = Label(self.window, text="", 
                           width = 80, height = 6, font = 15)
        self.label_result_title = Label(self.window, text="", fg="blue",
                           width = 80, height = 2, font = 15)        
        self.text_area = scrolledtext.ScrolledText(self.window, 
                           width = 80, height = 30, font = 10)
        self.entry = Entry(self.window,
                           width = 60, font = 15)
        self.button = Button(
            self.window, text="Поиск", command = self.print_request)
        
        
        self.label.grid(
            row=0, column = 0, columnspan = 2)
        self.entry.grid(
            row=1, column = 0)
        self.button.grid(
            row=1, column = 1)        
        self.label_request_title.grid(
            row=2, column = 0)
        self.label_request.grid(
            row=3, column = 0)        
        self.label_result_title.grid(
            row=4, column = 0)        
        self.text_area.grid(
            row=5, column = 0, pady = 20, padx = 20)        
      
    
    def print_request(self):
        
        key_word_string = self.entry.get()
        
        self.entry.delete(0, 'end')
        self.text_area.delete(0.0, 'end')
        
        self.label_request_title['text'] = f"Поиск по запросу:"
        
        key_word_list = key_word_string.lower().split(", ")
        key_word_print = '\n'.join(key_word_list)
        self.label_request['text'] =  f"{key_word_print}" 
        
        book_count, text_word_context = (
            Librarian(key_word_list, self.book_content, self.book_content_lower).text_context())        
        
        self.label_result_title['text'] = f"Результат поиска: {book_count}"        
        self.text_area.insert(INSERT, f"{text_word_context}")

In [241]:
class Reader:
    
    def __init__(self):
        abs_path = path.join(getcwd(), "books")
        self.book_list = listdir(abs_path)
        self.book_content = self.file_content(self.book_list)
        
        
    def file_content(self, book_list):
        content_dict = {}
        content_dict_lower = {}
        for file_name in tqdm(self.book_list):            
            content_dict[file_name], content_dict_lower[file_name] = self._file_read(file_name)
        return content_dict, content_dict_lower           
    
       
    def _file_read(self, file_name):
        abs_path = path.join(getcwd(), f"books\\{file_name}")
        try:
            with open(abs_path, "r") as file:
                content = file.read().replace("\n", " ")
        except UnicodeDecodeError:
            with open(abs_path, "r", encoding="UTF-8") as file:
                content = file.read().replace("\n", " ")
        content_lower = content.lower()
        return content, content_lower

In [257]:
class Librarian:
    
    def __init__(self, key_word_list, book_content, book_content_lower):
        self.key_word_list = key_word_list
        self.book_content = book_content
        self.book_content_lower = book_content_lower
    
    
    def _file_search(self):        
        book_dict_find = {}
        for book, content in self.book_content_lower.items():
            key_word_list_find = []
            index_list = []
            flag = True
            for word in self.key_word_list:
                if flag == True:
                    i = content.find(word)
                    if i != -1:
                        key_word_list_find.append(word)
                        index_list.append(i)
                    else:
                        flag = False
            if self.key_word_list == key_word_list_find:
                book_dict_find[book] = index_list
        return book_dict_find
    
    def text_context(self):
        book_dict_find = self._file_search()
        
        book_count = len(book_dict_find)
        
        text_word_context = ""
        dict_out = defaultdict(str) 
        for book, index_list in book_dict_find.items():
            len_book_content = len(self.book_content[book])
            len_word_context = 50
            for i in index_list:
                if len_book_content < (2 * len_word_context + 1):
                    dict_out[book] += (
                        f"... {self.book_content[book]} ... -+++- ")
                elif len_book_content - i < (2 * len_word_context):
                    dict_out[book] += (
                        f"... {self.book_content[book][i - len_word_context:]} ... -+++- ")                
                elif i > len_word_context:
                    dict_out[book] += (
                        f"... {self.book_content[book][(i - len_word_context):(i + len_word_context)]} ... -+++- ")
                else:
                    dict_out[book] += (
                        f"... {self.book_content[book][:(i + len_word_context)]} ... -+++- ")
            text_word_context += f"{book}\n{dict_out[book]}\n\n" 
        return book_count, text_word_context

In [260]:
# Librarian ---  тесты на время для различных запросов (запрос -- список в нижнем регистре)

file_content, file_content_lower = Reader().book_content


s1 = ["что"]
s2 = ["чехов", "агафья"]
s6 = ["что", "кто", "как", "и", "человек", "барышне", "барыне", "решительно", "надя"]
s7 = ["и чем кончился роман шамохина - мне неизвестно."]
s8 = ["что кухарке действительно лучше жить", "нежели барыне или барышне."]
s9 = ["book"]

print(s1)
start_time = datetime.now()
print(Librarian(s1, file_content, file_content_lower).text_context()[0])
print(datetime.now() - start_time)
print()

print(s2)
start_time = datetime.now()
print(Librarian(s2, file_content, file_content_lower).text_context()[0])
print(datetime.now() - start_time)
print()

print(s6)
start_time = datetime.now()
print(Librarian(s6, file_content, file_content_lower).text_context()[0])
print(datetime.now() - start_time)
print()

print(s7)
start_time = datetime.now()
print(Librarian(s7, file_content, file_content_lower).text_context())
print(datetime.now() - start_time)
print()

print(s8)
start_time = datetime.now()
print(Librarian(s8, file_content, file_content_lower).text_context()[0])
print(datetime.now() - start_time)
print()

print(s9)
start_time = datetime.now()
print(Librarian(s9, file_content, file_content_lower).text_context())
print(datetime.now() - start_time)
print()


['что']
666
0:00:00.005987

['чехов', 'агафья']
4
0:00:00.069810

['что', 'кто', 'как', 'и', 'человек', 'барышне', 'барыне', 'решительно', 'надя']
5
0:00:00.116688

['и чем кончился роман шамохина - мне неизвестно.']
(1, 'Chekhov__Ariadna.txt\n...  другой день после этой встречи я выехал из Ялты, и чем кончился роман Шамохина - мне неизвестно. ... -+++- \n\n')
0:00:00.029921

['что кухарке действительно лучше жить', 'нежели барыне или барышне.']
1
0:00:00.042394

['book']
(1, 'V__________S______________B.txt\n... Very short book. ... -+++- \n\n')
0:00:00.036902

